# AI Assignment 2

In [1]:
import numpy as np

np.set_printoptions(threshold=np.inf, linewidth=300)
import time

import pandas as pd
from PIL import Image
import heapq

In [2]:
class Map_Obj():
    def __init__(self, task=1):
        self.start_pos, self.goal_pos, self.end_goal_pos, self.path_to_map = self.fill_critical_positions(
            task)
        self.int_map, self.str_map = self.read_map(self.path_to_map)
        self.tmp_cell_value = self.get_cell_value(self.goal_pos)
        self.set_cell_value(self.start_pos, ' S ')
        self.set_cell_value(self.goal_pos, ' G ')
        self.tick_counter = 0
        self.path = []

    def read_map(self, path):
        """
        Reads maps specified in path from file, converts them to a numpy array and a string array. Then replaces
        specific values in the string array with predefined values more suitable for printing.
        :param path: Path to .csv maps
        :return: the integer map and string map
        """
        # Read map from provided csv file
        df = pd.read_csv(path, index_col=None,
                         header=None)  #,error_bad_lines=False)
        # Convert pandas dataframe to numpy array
        data = df.values
        # Convert numpy array to string to make it more human readable
        data_str = data.astype(str)
        # Replace numeric values with more human readable symbols
        data_str[data_str == '-1'] = ' # '
        data_str[data_str == '1'] = ' . '
        data_str[data_str == '2'] = ' , '
        data_str[data_str == '3'] = ' : '
        data_str[data_str == '4'] = ' ; '
        return data, data_str

    def fill_critical_positions(self, task):
        """
        Fills the important positions for the current task. Given the task, the path to the correct map is set, and the
        start, goal and eventual end_goal positions are set.
        :param task: The task we are currently solving
        :return: Start position, Initial goal position, End goal position, path to map for current task.
        """
        if task == 1:
            start_pos = [27, 18]
            goal_pos = [40, 32]
            end_goal_pos = goal_pos
            path_to_map = 'Samfundet_map_1.csv'
        elif task == 2:
            start_pos = [40, 32]
            goal_pos = [8, 5]
            end_goal_pos = goal_pos
            path_to_map = 'Samfundet_map_1.csv'
        elif task == 3:
            start_pos = [28, 32]
            goal_pos = [6, 32]
            end_goal_pos = goal_pos
            path_to_map = 'Samfundet_map_2.csv'
        elif task == 4:
            start_pos = [28, 32]
            goal_pos = [6, 32]
            end_goal_pos = goal_pos
            path_to_map = 'Samfundet_map_Edgar_full.csv'
        elif task == 5:
            start_pos = [14, 18]
            goal_pos = [6, 36]
            end_goal_pos = [6, 7]
            path_to_map = 'Samfundet_map_2.csv'

        return start_pos, goal_pos, end_goal_pos, path_to_map

    def get_cell_value(self, pos):
        return self.int_map[pos[0], pos[1]]

    def get_goal_pos(self):
        return self.goal_pos

    def get_start_pos(self):
        return self.start_pos

    def get_end_goal_pos(self):
        return self.end_goal_pos

    def get_maps(self):
        # Return the map in both int and string format
        return self.int_map, self.str_map

    def move_goal_pos(self, pos):
        """
        Moves the goal position towards end_goal position. Moves the current goal position and replaces its previous
        position with the previous values for correct printing.
        :param pos: position to move current_goal to
        :return: nothing.
        """
        tmp_val = self.tmp_cell_value
        tmp_pos = self.goal_pos
        self.tmp_cell_value = self.get_cell_value(pos)
        self.goal_pos = [pos[0], pos[1]]
        self.replace_map_values(tmp_pos, tmp_val, self.goal_pos)

    def set_cell_value(self, pos, value, str_map=True):
        if str_map:
            self.str_map[pos[0], pos[1]] = value
        else:
            self.int_map[pos[0], pos[1]] = value

    def print_map(self, map_to_print):
        # For every column in provided map, print it
        for column in map_to_print:
            print(column)

    def pick_move(self):
        """
        A function used for moving the goal position. It moves the current goal position towards the end_goal position.
        :return: Next coordinates for the goal position.
        """
        if self.goal_pos[0] < self.end_goal_pos[0]:
            return [self.goal_pos[0] + 1, self.goal_pos[1]]
        elif self.goal_pos[0] > self.end_goal_pos[0]:
            return [self.goal_pos[0] - 1, self.goal_pos[1]]
        elif self.goal_pos[1] < self.end_goal_pos[1]:
            return [self.goal_pos[0], self.goal_pos[1] + 1]
        else:
            return [self.goal_pos[0], self.goal_pos[1] - 1]

    def replace_map_values(self, pos, value, goal_pos):
        """
        Replaces the values in the two maps at the coordinates provided with the values provided.
        :param pos: coordinates for where we want to change the values
        :param value: the value we want to change to
        :param goal_pos: The coordinate of the current goal
        :return: nothing.
        """
        if value == 1:
            str_value = ' . '
        elif value == 2:
            str_value = ' , '
        elif value == 3:
            str_value = ' : '
        elif value == 4:
            str_value = ' ; '
        else:
            str_value = str(value)
        self.int_map[pos[0]][pos[1]] = value
        self.str_map[pos[0]][pos[1]] = str_value
        self.str_map[goal_pos[0], goal_pos[1]] = ' G '

    def tick(self):
        """
        Moves the current goal position every 4th call if current goal position is not already at the end_goal position.
        :return: current goal position
        """
        # For every 4th call, actually do something
        if self.tick_counter % 4 == 0:
            # The end_goal_pos is not set
            if self.end_goal_pos is None:
                return self.goal_pos
            # The current goal is at the end_goal
            elif self.end_goal_pos == self.goal_pos:
                return self.goal_pos
            else:
                # Move current goal position
                move = self.pick_move()
                self.move_goal_pos(move)
                #print(self.goal_pos)
        self.tick_counter += 1

        return self.goal_pos

    def set_start_pos_str_marker(self, start_pos, map):
        # Attempt to set the start position on the map
        if self.int_map[start_pos[0]][start_pos[1]] == -1:
            self.print_map(self.str_map)
            print('The selected start position, ' + str(start_pos) +
                  ' is not a valid position on the current map.')
            exit()
        else:
            map[start_pos[0]][start_pos[1]] = ' S '

    def set_goal_pos_str_marker(self, goal_pos, map):
        # Attempt to set the goal position on the map
        if self.int_map[goal_pos[0]][goal_pos[1]] == -1:
            self.print_map(self.str_map)
            print('The selected goal position, ' + str(goal_pos) +
                  ' is not a valid position on the current map.')
            exit()
        else:
            map[goal_pos[0]][goal_pos[1]] = ' G '
            

    def show_map(self, map=None):
        """
        A function used to draw the map as an image and show it.
        :param map: map to use
        :return: nothing.
        """
        # If a map is provided, set the goal and start positions
        if map is not None:
            self.set_start_pos_str_marker(self.start_pos, map)
            self.set_goal_pos_str_marker(self.goal_pos, map)
        # If no map is provided, use string_map
        else:
            map = self.str_map

        # Define width and height of image
        width = map.shape[1]
        height = map.shape[0]
        # Define scale of the image
        scale = 20
        # Create an all-yellow image
        image = Image.new('RGB', (width * scale, height * scale),
                          (255, 255, 0))
        # Load image
        pixels = image.load()

        # Define what colors to give to different values of the string map (undefined values will remain yellow, this is
        # how the yellow path is painted)
        colors = {
            ' # ': (211, 33, 45),
            ' . ': (215, 215, 215),
            ' , ': (166, 166, 166),
            ' : ': (96, 96, 96),
            ' ; ': (36, 36, 36),
            ' S ': (255, 0, 255),
            ' G ': (0, 128, 255)
        }
        # Go through image and set pixel color for every position
        for y in range(height):
            for x in range(width):
                if map[y][x] not in colors: continue
                for i in range(scale):
                    for j in range(scale):
                        pixels[x * scale + i,
                               y * scale + j] = colors[map[y][x]]
                        
        ###Extra code for visualizing path
        for pos in self.path:
            for i in range(scale):
                    for j in range(scale):
                        pixels[pos[1] * scale + i,
                               pos[0] * scale + j] = (0, 0, 255)
        
        # Show image
        image.show()
    
            

### My code

### Pseudo kode

Starter på en posisjon. Lager node for alle posisjoner rundt. En node setter deretter parent, koordinatet, g,h,f som medlemsvariabler. Prioriteten bestemmes av distanse til mål + vekten.
while priolist:
    hent ut prionode og set den som current
    get nodene rundt current node
        if goal: set goal = true, winning node = node
        if in closed list: skip
        if in open list: uppdate g, h, f if one is lower when reached from current node
        else: Initialize it
        insert in prioqueue with f as prio
    insert current node in closed list
    if goal: finnished
    
Merk: Noder karakteriseres fullstendig med sin posisjon, slik at ved sammenlikning av nodere er det posisjonene som sammelignes.

##### Class: node
Creates a node objekt corresponding to each position in the map grid. The node is therefore uniquely defined by its position coordinates, together with its parent to allow backtracking, weight from starting position g, its heuristic h which is the 1-norm between its position and the goal, and the negative priority f which is the sum of h and g. The g,h,f and the corresponding parent is continuisly changed, as we only care about the cheapest path to given position.

In [3]:
class node():
    def __init__(self, pos, parent = None):
        self.pos = pos
        self.parent = parent
        self.g = 0
        self.f = 1000 #placeholder
        self.h = 1000 #placeholder
#     self.g = self.calc_g() #g = distance from start
#     self.f = self.calc_f() #f = priority
#     self.h = self.calc_h() #h = heuristic
    def calc_g(self, parent,w = 1):
        if parent == None:
            return 0
        return parent.get_g() + w
    def calc_h(self,goal_position):
        pos = self.get_pos()
        x = pos[1]
        y = pos[0]
        return abs(x - goal_position[1]) + abs(y - goal_position[0]) #uniform metric
    def calc_f():
        return self.get_g() + self.get_h()
    def get_g(self):
        return self.g
    def get_h(self):
        return self.h
    def get_f(self):
        return self.f
    def get_pos(self):
        return self.pos
    def get_parent(self):
        return self.parent
    def set_g(self,dist):
        self.g = dist
    def set_h(self,dist):
        self.h = dist
    def set_f(self,dist):
        self.f = dist
    def set_ghf(self,gval,hval,fval):
        self.g = gval
        self.h = hval
        self.f = fval
    def set_parent(self,new_parent):
        self.parent = new_parent
        

##### Function: find_children_not_in_closed(current,closed_list, maps)
Creates potential nodes in all positions 1 lenght from the current node. It automaticly skips positions of nodes in the closed list and itself. 

In [4]:
def find_children_not_in_closed(current,closed_list, maps):
    candidates = []
    #print("printing current before steps: ", current.get_pos())
    for i in [-1,0,1]:
            for j in [-1,0,1]:
                if (i == j and i == 0):
                    #print("skippet itself")
                    continue #To not add itself.
                else:
                    skip = False
                    potential_pos = current.get_pos()#+[i,j]
                    new_x = potential_pos[0] + i
                    new_y = potential_pos[1] + j
                    potential_pos = [new_x,new_y]
                    check_for_wall = maps.get_cell_value(potential_pos)
                    if check_for_wall == -1:
                        #print("skipped wall")
                        continue
                    #print("new pos is: ", potential_pos)
                    for pos in closed_list: #closed list er full av positions
                        if pos.get_pos()  == potential_pos:
                            skip = True
                            #print("skipped closed pos")
                    if skip:
                        continue #We don't want to add nodes which is in the closed list.
                    child_not_in_closed = node(potential_pos,current)
                    candidates.append(child_not_in_closed)
                    #print("added: ", child_not_in_closed.get_pos(), "to the candidate list.")
    return candidates

##### Function: insert_with_prio(array,cand)
Used when adding new nodes to the open list to ensure the open list stays sorted with regards to priority f.

In [5]:
def insert_with_prio(array,cand):
    new_copy = array.copy()
    if len(array) == 0:
        array.append(cand)
        #print("open list was 0")
        return array
    prio = cand.get_f()
    #print("this is the prio of new node: ", prio)
    if prio < array[0].get_f():
            array.insert(0, cand)
            
    for count,item in enumerate(array):
#         print("this is the prio of node in open: ", item.get_f())
#         print("and the position is ", item.get_pos())
        if prio > item.get_f():
            if count == (len(array)-1):
                array.append(cand)
            continue
        array.insert(count, cand) #Inserts right before the first element with less prio
#         print("inserted")
        break
        

    if array == new_copy:
        print("ERROR: ")
#     print(prio, '\t', array[-1].get_f())
#     print("len array: ", len(array))
    
    return array

##### Function:  get_final_path(current)
Takes the node given in "current" and iterate through parent nodes until it reaches the starting position. In every step it appends the nodes possition into a list which it returns. This is used with the node on the goal position to get the array of positions of the path taken to reach the goal.


In [6]:
def get_final_path(current):
    pos = current.get_pos()
    final_path = []
    g = 10
    while g:
        parent = current.get_parent()
        final_path.append(parent.get_pos())
        g = parent.get_g()
        current = parent
    return final_path

In [7]:
def Astar(A):
    goal_pos = A.get_goal_pos()
    start_pos = A.get_start_pos()
    print("start pos: ", start_pos, '\n')
    print("goal pos: ", goal_pos, '\n')
    start_node = node(start_pos)
    start_node.set_ghf(0,0,0)
    #print("startnode har g= ", start_node.get_g())
    #heapq.heappush(open_list,(start_node.get_f(),start_node))
    open_list = [start_node]# prioQueue()
    closed_list = []
    count = 0
    while len(open_list) > 0:
        count +=1
#         max_prio = 1000
#         for item in open_list:
#             if item.get_f() < max_prio:
#                 current = item
        #print("before pop: ", len(open_list))
        current = open_list.pop(0) #Here we presume the open list is sortet by priority from lowest to highest
        #print("after pop: ", len(open_list))
        #print("current: ",current.get_pos())
        closed_list.append(current) #Maybe better in the end of the iteration
        candidates = find_children_not_in_closed(current,closed_list, A)
        #print("len candidates is: ", len(candidates))
        open_list_copy = open_list 
        
        
        for candidate in candidates:
            candidate_pos = candidate.get_pos()
            if candidate_pos == goal_pos:
                print("goal reached!")
                path_to_goal = get_final_path(candidate)
                return path_to_goal
            w = A.get_cell_value(candidate_pos)
            g = candidate.calc_g(current, w)
#             print("g= ", g)
#             print("w= ", w)
#             print("parents g = ", (candidate.get_parent()).get_g())
            h = candidate.calc_h(goal_pos)
            f = g + h
            candidate.set_ghf(g,h,f)
#             print("g,h,f = ", g,h,f)
#             print("candidate pos: ", candidate.get_pos())
            found = False
            #print(open_list, "<- sjekker open list")
            for node_open in open_list_copy: #Looking only in the list of coordinates for uniqueness
                #node_open = open_list_copy[node_it]
                #print(candidate_pos, " != ", node_open.get_pos())
                if candidate_pos == node_open.get_pos():
#                     print("duplicate found")
#                     print(node_open.get_pos())
                    found = True
                    #if found: print("found is true")
                    if node_open.get_f() > f:
                        node_open.set_ghf(g,h,f)
                        node_open.set_parent(current)
                        
#                         print("replaced")
                    break
            if found != True: 
#                     print("open list before inserting: ", len(open_list))
                    open_list = insert_with_prio(open_list,candidate)
#                     print("did insert candidate with pos: ", candidate.get_pos())
#                     for i in open_list:
#                         print(i.get_f())
        
        if count > 1000:
            print("count limit reached")
            break
        if len(open_list) == 0:
            print("Open list is empty")
        #print(len(open_list))
    

##### Running codes
Here, all the tasks are ran through the A* algorithm, and the path taken to reached the goal is then printed as an overwritten map with visited nodes colored blue.

In [8]:
maps1 = Map_Obj()
finale_path1 = Astar(maps1)

start pos:  [27, 18] 

goal pos:  [40, 32] 

goal reached!


In [9]:
maps2 = Map_Obj(task = 2)
finale_path2 = Astar(maps2)

start pos:  [40, 32] 

goal pos:  [8, 5] 

goal reached!


In [10]:
maps3 = Map_Obj(task = 3)
finale_path3 = Astar(maps3)

start pos:  [28, 32] 

goal pos:  [6, 32] 

goal reached!


In [11]:
maps4 = Map_Obj(task = 4)
finale_path4 = Astar(maps4)

start pos:  [28, 32] 

goal pos:  [6, 32] 

goal reached!


In [12]:
maps1.path = finale_path1
maps1.show_map()

In [13]:
maps2.path = finale_path2
maps2.show_map()

In [14]:
maps3.path = finale_path3
maps3.show_map()

In [15]:
maps4.path = finale_path4
maps4.show_map()

##### Easy changes can be made to the code to make the protagonist not clip through stairwalls etc if that is imposed as an additional criteria for the program.